# More Earth Engine: Irrigation, Snow, and Soils Data Extraction

In this notebook, we will:
1. Use the `sample_snodas_swe` function to perform snodas snow water equivalent (SWE) extraction.
2. Use the `get_irrigation` function to perform irrigation mask extraction.
3. Use the `get_ssurgo` function to perform soils extraction.

Ensure that the Earth Engine Python API is authenticated and configured, and that the Earth Engine CLI is available in your environment.

## 1. Import Libraries and Authorize Earth Engine

In [1]:
import os
import sys
import ee

# append the project path to the environment (mine is in /home/dgketchum/PycharmProjects')
root = '/home/dgketchum/PycharmProjects/swim-rs'
sys.path.append(root)

from data_extraction.ee.snodas_export import sample_snodas_swe
from data_extraction.snodas.snodas import create_timeseries_json
from data_extraction.ee.ee_props import get_irrigation, get_ssurgo
from data_extraction.ee.ee_utils import is_authorized
from utils.google_bucket import list_and_copy_gcs_bucket

sys.path.insert(0, os.path.abspath('../..'))
sys.setrecursionlimit(5000)

if not is_authorized():
    ee.Authenticate()
ee.Initialize()

Authorized


## 1: Extract Raster Data

Here, we're going to use functions with a similar workflow to those we used to get ETf and NDVI data from Earth Engine.

For the raster data extraction, there are three functions we need to run:

*   **`sample_snodas_swe`**:
    *   This function iterates over the daily SNODAS images in Earth Engine, extracting mean SWE for each sample plot for each day, September through May.
    *   https://nsidc.org/data/g02158/versions/1
    *   We will follow up and write the SNODAS data to per-field files with `snodas_processing`.
*   **`get_irrigation`**:
    *   This function uses IrrMapper to get statistics about the irrigation status of each plot for each year, including the fraction of the plot that was irrigated.
    *   https://www.mdpi.com/2072-4292/12/14/2328
*   **`get_ssurgo`**:
    *   This function uses data summarized and put in a public Earth Engine asset by Charles Morton at Desert Research Institute from SSURGO (https://www.nrcs.usda.gov/resources/data-and-reports/soil-survey-geographic-database-ssurgo) to summarize plot-scale soil texture and hydraulic properties used by SWIM-RS.
 
Note: The module carrying `get_irrigation` and `get_ssurgo` also has functions for extracting vegetation height (useful for modeling natural/perennial vegetated sample plots; `get_landfire`), and for crop type (useful for sampling management scenarios; `get_cdl`). These have the same function call argruments and the same output format as with `get_irrigation` or `get_ssurgo`.

In [2]:
ee_account = 'ee-dgketchum'

# Define Constants and Remote Sensing Data Paths
# TODO: remove hard-coded collecitons and use variables defined here
SWE = 'projects/earthengine-legacy/assets/projects/climate-engine/snodas/daily'

bucket = 'wudr'
fields = 'projects/ee-dgketchum/assets/swim/mt_sid_boulder'

command = '/home/dgketchum/google-cloud-sdk/bin/gsutil'

# We must specify which column in the shapefile represents the field's unique ID, in this case it is 'FID_1'
FEATURE_ID = 'FID_1'

### SWE Data

Notes
- Ensure that your Cloud Storage bucket has the correct permissions for Earth Engine to write to it.
- This will produce a monthly dataset for Sep - May, regardless of SWE status at the sample plots.

In [3]:
# let's send all the data we get to our tutorial directories
snow_dst = os.path.join(root, 'tutorial/step_2_earth_engine_extract/snodas/extracts')
if not os.path.isdir(snow_dst):
    os.mkdir(snow_dst)

In [ ]:
# Here, we run the sample_snodas_swe function on the uploaded asset.
# Note we use 'check_dir' to check if it's already written to the directory, and 'overwrite=False' so we don't write it again if it is.
sample_snodas_swe(fields, bucket, debug=False, check_dir=snow_dst, overwrite=False, feature_id=FEATURE_ID)

Copy the data to the local machine once the processing is complete, as with the remote sensing data.

In [4]:
glob_ = 'swe'
list_and_copy_gcs_bucket(command, bucket, snow_dst, glob=glob_, dry_run=False, overwrite=False)

/home/dgketchum/PycharmProjects/swim-rs/tutorial/step_2_earth_engine_extract/snodas/extracts/swe_2004_01.csv exists, skipping
/home/dgketchum/PycharmProjects/swim-rs/tutorial/step_2_earth_engine_extract/snodas/extracts/swe_2004_02.csv exists, skipping
/home/dgketchum/PycharmProjects/swim-rs/tutorial/step_2_earth_engine_extract/snodas/extracts/swe_2004_03.csv exists, skipping
/home/dgketchum/PycharmProjects/swim-rs/tutorial/step_2_earth_engine_extract/snodas/extracts/swe_2004_04.csv exists, skipping
/home/dgketchum/PycharmProjects/swim-rs/tutorial/step_2_earth_engine_extract/snodas/extracts/swe_2004_05.csv exists, skipping
/home/dgketchum/PycharmProjects/swim-rs/tutorial/step_2_earth_engine_extract/snodas/extracts/swe_2004_06.csv exists, skipping
/home/dgketchum/PycharmProjects/swim-rs/tutorial/step_2_earth_engine_extract/snodas/extracts/swe_2004_07.csv exists, skipping
/home/dgketchum/PycharmProjects/swim-rs/tutorial/step_2_earth_engine_extract/snodas/extracts/swe_2004_08.csv exists, s

We will go ahead and convert the month-by-month files to a per-field daily time series of SWE that we will need in Step 4.

In [4]:
snow_out = os.path.join(root, 'tutorial/step_2_earth_engine_extract/snodas/snodas.json')
create_timeseries_json(snow_dst, snow_out, feature_id='FID_1')

Processing CSV files: 100%|██████████████████| 240/240 [00:01<00:00, 232.27it/s]


wrote /home/dgketchum/PycharmProjects/swim-rs/tutorial/step_2_earth_engine_extract/snodas/snodas.json


### Irrigation Data

Notes
- This will produce an annual dataset of the IrrMapper-estimated irrigated fraction for each sample plot.

In [3]:
description = 'tutorial_irr'
get_irrigation(fields, description, debug=False, selector=FEATURE_ID)

In [4]:
dst = os.path.join(root, 'tutorial/step_2_earth_engine_extract/properties')
list_and_copy_gcs_bucket(command, bucket, dst, glob=description, dry_run=False, overwrite=True)

Copying gs://wudr/tutorial_irr.csv: Copying gs://wudr/tutorial_irr.csv...
/ [1 files][ 51.9 KiB/ 51.9 KiB]                                                
Operation completed over 1 objects/51.9 KiB.                                     

Copied gs://wudr/tutorial_irr.csv to /home/dgketchum/PycharmProjects/swim-rs/tutorial/step_2_earth_engine_extract/properties


### Soils Data

Notes
- This will produce an single dataset of the SSURGO-estimated soil properties fraction for each sample plot.

In [7]:
description = 'tutorial_ssurgo'
get_ssurgo(fields, description, debug=False, selector=FEATURE_ID)

tutorial_ssurgo


In [9]:
dst = os.path.join(root, 'tutorial/step_2_earth_engine_extract/properties')
list_and_copy_gcs_bucket(command, bucket, dst, glob=description, dry_run=False, overwrite=False)

Copying gs://wudr/tutorial_ssurgo.csv: Copying gs://wudr/tutorial_ssurgo.csv...
/ [1 files][  6.7 KiB/  6.7 KiB]                                                
Operation completed over 1 objects/6.7 KiB.                                      

Copied gs://wudr/tutorial_ssurgo.csv to /media/research/IrrigationGIS/swim/examples/tutorial/properties
